In [1]:
import pygame as py
from tkinter import *
from tkinter import messagebox
import random

# Class of game operative logic and methods
class Game:
    # initialization process
    def __init__(self, gamepanel):
        self.gamepanel = gamepanel
        self.end = False
        self.won = False
 
    def start(self):
        # ramdomly appear two cells (2 or 4)
        self.gamepanel.random_cell()
        self.gamepanel.random_cell()
        
        self.gamepanel.paint_grid()
        self.gamepanel.window.bind('<Key>', self.event_handlers) # define 4 <Key> signals
        self.gamepanel.window.mainloop()
 
    # realize the rule of the game
    def event_handlers(self, event):
        if self.end or self.won:
            return
 
        self.gamepanel.compress = False # remove the orginal block
        self.gamepanel.merge = False # merge a new block of sum of two blocks
        self.gamepanel.moved = False # change the location to new condition
 
        enter_key = event.keysym #connect the unique symbol to each key on the keyboard. 
 
        if enter_key == 'Up':
            self.gamepanel.transpose() # respose the grid 
            self.gamepanel.drifting_left() # all nonzeros to left
            self.gamepanel.merge_grid() # merge grid
            self.gamepanel.moved = self.gamepanel.compress or self.gamepanel.merge # update the current condition
            self.gamepanel.drifting_left() # all non zeros to left
            self.gamepanel.transpose() # return the normal direction
 
        # For example of process enter_key == 'Down' (before a new random block appears):
        # | 0 4 0 2 | -> | 0 8 4 2 | -> | 2 4 8 0 | -> | 2 4 8 0 | -> | 2 4 8 0 | -> | 0 8 4 2 | -> | 0 0 0 0 |
        # | 8 0 2 2 |    | 4 0 4 0 |    | 0 4 0 4 |    | 4 4 0 0 |    | 8 0 0 0 |    | 0 0 0 8 |    | 8 0 0 0 |
        # | 4 4 0 0 |    | 0 2 0 4 |    | 4 0 2 0 |    | 4 2 0 0 |    | 4 2 0 0 |    | 0 0 2 4 |    | 4 0 2 2 |
        # | 2 0 4 2 |    | 2 2 0 2 |    | 2 0 2 2 |    | 2 2 2 0 |    | 4 2 0 0 |    | 0 0 2 4 |    | 2 8 4 4 |
        # original mtx.  transpose()    reverse()     drifting_left() merge_grid()   reverse()      transpose()
        # the result is the same as the theoretical answer, which is correct
        elif enter_key == 'Down':
            self.gamepanel.transpose()
            self.gamepanel.reverse()
            self.gamepanel.drifting_left()
            self.gamepanel.merge_grid()
            self.gamepanel.moved = self.gamepanel.compress or self.gamepanel.merge
            self.gamepanel.drifting_left()
            self.gamepanel.reverse()
            self.gamepanel.transpose()
 
        elif enter_key == 'Left':
            self.gamepanel.drifting_left()
            self.gamepanel.merge_grid()
            self.gamepanel.moved = self.gamepanel.compress or self.gamepanel.merge
            self.gamepanel.drifting_left()
 
        elif enter_key == 'Right':
            self.gamepanel.reverse()
            self.gamepanel.drifting_left()
            self.gamepanel.merge_grid()
            self.gamepanel.moved = self.gamepanel.compress or self.gamepanel.merge
            self.gamepanel.drifting_left()
            self.gamepanel.reverse()
 
        self.gamepanel.paint_grid()
        # print out current score below the game panel
        score_show = self.gamepanel.score
        self.gamepanel.label_score['text'] = str(self.gamepanel.score) 
 
        flag = 0
        # judging winning whether get 2048 block
        for i in range(4):
            for j in range(4):
                if self.gamepanel.grid_cell[i][j] >= 2048:
                    messagebox.showinfo('2048', message='Congrats, you win')
                    print("Winner")
                    return
 
        for i in range(4):
            for j in range(4):
                if self.gamepanel.grid_cell[i][j] == 0:
                    flag = 1
                    break
                    
        # judging losing if there is no empty and merge process cannot be down at the same time
        if not (flag or self.gamepanel.can_merge()):
            self.end = True
            messagebox.showinfo('2048', 'Game Over!!!')
            print("Over")
        
        # add a new random block after each effective operation
        if self.gamepanel.moved:
            self.gamepanel.random_cell()
 
        self.gamepanel.paint_grid()
    

    
# Class of game board design and basic algorithms
class Board:
    # block background color
    bg_color = {
        '2': '#eee4da',
        '4': '#ede0c8',
        '8': '#ffe799',
        '16': '#ffdf78',
        '32': '#ffd477',
        '64': '#ffcc26',
        '128': '#c39600',
        '256': '#9b7600',
        '512': '#6a5100',
        '1024': '#006c9b',
        '2048': '#004461',
    }
    
    # number text color (can only appear in this game)
    color = {
        '2': '#776e65',
        '4': '#776e65',
        '8': '#776e65',
        '16': '#776e65',
        '32': '#776e65',
        '64': '#776e65',
        '128': '#f9f6f2',
        '256': '#f9f6f2',
        '512': '#f9f6f2',
        '1024': '#f9f6f2',
        '2048': '#f9f6f2',
    }
 
    # initialization process
    def __init__(self):
        self.n = 4
        self.window = Tk()
        self.window.title('2048')
        self.game_area = Frame(self.window, bg='azure3') # build game board window
        self.board = []
        self.grid_cell = [[0] * 4 for _ in range(4)] # 4 by 4 empty blocks matrix (all zeros)
        self.compress = False
        self.merge = False
        self.moved = False
        self.score = 0
        self.label_score = Label(self.game_area, text='0', font=("bold", 22, "bold"),bg="#bbada0", fg="#ffffff")
 
        for i in range(4):
            rows = []
            for j in range(4):
                l = Label(self.game_area, text='Score', bg='azure4',
                          font=('arial', 22, 'bold'), width=4, height=2)
                l.grid(row=i, column=j, padx=7, pady=7)
                rows.append(l)
            # show score below game board
            self.board.append(rows)
        # print out current score
        label = Label(self.game_area, text='Score', font=("bold", 20, "bold"),bg="#bbada0", fg="#eee4da")
        label.grid(row=4, column=0, padx=5, pady=5)
        self.label_score.grid(row=4, columnspan=2, column=1, padx=5, pady=5)
        self.game_area.grid()

    # reverse the matrix of game(right and left turn over)
    def reverse(self):
        for i in range(4):
            self.grid_cell[i].reverse()
 
    # operation of up and down (right up and left down turn over)
    def transpose(self):
        self.grid_cell = [list(t) for t in zip(*self.grid_cell)]
 
    # operation of shifting all blocks to left(compress empty blocks) 
    def drifting_left(self):
        self.compress = False
        temp = [[0] * 4 for _ in range(4)] # temp is a new 4 by 4 all zero matrix
        # traversing from left to right for each row, copy all nonzeros from left two right to temp
        # all zeros appears on right side
        for i in range(4):
            cnt = 0 # pointer
            for j in range(4):
                if self.grid_cell[i][j] != 0: # only copy nonzeros
                    temp[i][cnt] = self.grid_cell[i][j]
                    if cnt != j:
                        self.compress = True
                    cnt += 1 # after each copy, pointer point next
        self.grid_cell = temp # temp to be new grid cell
        
    # block merge rule
    def merge_grid(self):
        self.merge = False
        for i in range(4):
            for j in range(3):
                if self.grid_cell[i][j] == self.grid_cell[i][j + 1] and self.grid_cell[i][j] != 0:
                    # the second position of adjacent same-valur blocks times 2, empty out the first position
                    self.grid_cell[i][j] *= 2 
                    self.grid_cell[i][j + 1] = 0
                    # total score plus the same amount
                    self.score += self.grid_cell[i][j]
                    self.merge = True # activate merge function
        #add background music to each operation
        py.mixer.init()
        py.mixer.music.load('double.mp3') # here is macos version, use window recommend revise to (r'.\double.mp3') 
        py.mixer.music.play(0)
                    
    # Ramdomly create block 2 or 4
    def random_cell(self):
        i, j = random.choice([(i, j) for i in range(4) for j in range(4) if self.grid_cell[i][j] == 0])
        self.grid_cell[i][j] = random.choice([2,4])
 
    # judge whether two blocks can merge or not 
    def can_merge(self):
        for i in range(4):
            for j in range(3):
                if self.grid_cell[i][j] == self.grid_cell[i][j + 1]: # for columns
                    return True
 
        for i in range(3):
            for j in range(4):
                if self.grid_cell[i][j] == self.grid_cell[i + 1][j]: # for rows 
                    return True
        return False
 
    # distribute color to each grid
    def paint_grid(self):
        for i in range(4):
            for j in range(4):
                if self.grid_cell[i][j] == 0:
                    self.board[i][j].config(text='', bg='azure4')
                else:
                    self.board[i][j].config(text=str(self.grid_cell[i][j]),
                                            bg=self.bg_color.get(str(self.grid_cell[i][j])),
                                            fg=self.color.get(str(self.grid_cell[i][j])))
  

# game starter
if __name__ == "__main__":
    gamepanel = Board()
    game2048 = Game(gamepanel)
    game2048.start()


pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [18]:
# if no pygame and tkinter on your computer yet, run the following code row by row:
"""
pip install pygame
pip install tkinter
"""

'\npip install pygame\npip install tkinter\n'